In [16]:
import chess.pgn
import numpy as np
import torch
import chess
import random

In [17]:
def create_uci_labels():
    """
    Creates the labels for the universal chess interface into an array and returns them
    :return:
    """
    labels_array = []
    letters = ["a", "b", "c", "d", "e", "f", "g", "h"]
    numbers = ["1", "2", "3", "4", "5", "6", "7", "8"]
    promoted_to = ["q", "r", "b", "n"]

    for l1 in range(8):
        for n1 in range(8):
            destinations = (
                [(t, n1) for t in range(8)]
                + [(l1, t) for t in range(8)]
                + [(l1 + t, n1 + t) for t in range(-7, 8)]
                + [(l1 + t, n1 - t) for t in range(-7, 8)]
                + [
                    (l1 + a, n1 + b)
                    for (a, b) in [
                        (-2, -1),
                        (-1, -2),
                        (-2, 1),
                        (1, -2),
                        (2, -1),
                        (-1, 2),
                        (2, 1),
                        (1, 2),
                    ]
                ]
            )
            for (l2, n2) in destinations:
                if (l1, n1) != (l2, n2) and l2 in range(8) and n2 in range(8):
                    move = letters[l1] + numbers[n1] + letters[l2] + numbers[n2]
                    labels_array.append(move)
    for l1 in range(8):
        l = letters[l1]
        for p in promoted_to:
            labels_array.append(l + "2" + l + "1" + p)
            labels_array.append(l + "7" + l + "8" + p)
            if l1 > 0:
                l_l = letters[l1 - 1]
                labels_array.append(l + "2" + l_l + "1" + p)
                labels_array.append(l + "7" + l_l + "8" + p)
            if l1 < 7:
                l_r = letters[l1 + 1]
                labels_array.append(l + "2" + l_r + "1" + p)
                labels_array.append(l + "7" + l_r + "8" + p)
    return labels_array


uci_table = create_uci_labels()
uci_dict = {label: i for i, label in enumerate(uci_table)}

In [43]:
import multiprocessing as mp
import torch.multiprocessing as tmp

In [47]:
def createStateObj(board):
    # convert state into a 8x8x12 tensor
    state = torch.zeros((18, 8, 8), dtype=torch.float32)
    net_piece_value = 0

    for square, piece in board.piece_map().items():
        for square, piece in board.piece_map().items():
            net_piece_value += piece_values[str(piece)]
            if piece.color == chess.WHITE:
                state[piece.piece_type - 1, square // 8, square % 8] = 1.0
            else:
                state[piece.piece_type + 5, square // 8, square % 8] = 1.0

    # append the 5 states above
    p1_can_castle_queenside = board.has_queenside_castling_rights(chess.WHITE)
    p1_can_castle_kingside = board.has_kingside_castling_rights(chess.WHITE)
    p2_can_castle_queenside = board.has_queenside_castling_rights(chess.BLACK)
    p2_can_castle_kingside = board.has_kingside_castling_rights(chess.BLACK)
    turn = board.turn

    state[12, :, :] = float(p1_can_castle_queenside)
    state[13, :, :] = float(p1_can_castle_kingside)
    state[14, :, :] = float(p2_can_castle_queenside)
    state[15, :, :] = float(p2_can_castle_kingside)
    state[16, :, :] = float(turn)
    state[17, :, :] = float(net_piece_value)

    return state


tag = ("1/2-1/2", "1-0", "0-1")


def createDataHelper(game):
    data = []
    X = []
    y = []

    if game.headers["Result"] == "1/2-1/2":
        win = 0.0
    elif game.headers["Result"] == "1-0":
        win = 1.0
    elif game.headers["Result"] == "0-1":
        win = -1.0
    else:
        print("Error: Unexpected result string" + game.headers["Result"])
        return []
    if game.headers["Result"] not in tag:
        print("Error: Unexpected result string" + game.headers["Result"])
        return []

    board = game.board()

    lst = list(game.mainline_moves())
    for move in lst[:-1]:
        X.append(createStateObj(board))
        board.push(move)
        y_ele = torch.zeros(1968, dtype=torch.float32)
        try:
            y_ele[uci_dict[str(move)]] = 1.0
        except KeyError:
            X.pop()
            break
        y.append(y_ele)

    for i in range(len(X)):
        data.append((X[i], y[i], torch.tensor(win, dtype=torch.float32)))

    return data


def createData(fp, n_data=200):
    data = []

    for i in range(n_data):
        game = chess.pgn.read_game(fp)
        data += createDataHelper(game)

    random.shuffle(data)
    X, y, win = zip(*data)
    return X, y, win


piece_values = {
    "p": 1,
    "n": 3,
    "b": 3,
    "r": 5,
    "q": 9,
    "k": 1000,
    "P": -1,
    "N": -3,
    "B": -3,
    "R": -5,
    "Q": -9,
    "K": -1000,
}

In [49]:
with open("../chess_engine/data/filtered/db2023.pgn") as fp:
    X, y, win = createData(fp)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'createDataHelper' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    sel

KeyboardInterrupt: 

In [ ]:
with open("../chess_engine/data/filtered/db2023.pgn") as fp:
    for i in range(200):
        game = chess.pgn.read_game(fp)

In [55]:
import torch
import torch.multiprocessing as mp

def square(x, queue):
    result = []
    for i in range(len(x)):
        result.append(x[i] ** 2)
    queue.put(result)

x = torch.tensor([1, 2, 3, 4, 5])
queue = mp.Queue()
p = mp.Process(target=square, args=(x, queue))
p.start()
output = queue.get()
p.join()
print(output)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'square' on <module '__main__' (built-in)>


KeyboardInterrupt: 